In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.7MB/s 


In [2]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK
from eli5.sklearn import PermutationImportance


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars/"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars


In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

Feature engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list):
    continue

  factorize_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorize_values
  else:
    df[feat + SUFFIX_CAT] = factorize_values

In [0]:
df['param_rok-produkcji__int'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc__int']  = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa__int'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' cm3')[0].replace(' ', '')))
df['param_przebieg__int'] = df['param_przebieg'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' km')[0].replace(' ', '')))

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
feats = ['param_przebieg__int', 'param_fuel-type__cat', 'seller_type__cat', 'feature_klimatyzacja-dwustrefowa', 'param_napęd__cat', 'param_faktura-vat__cat', 'param_stan__cat', 'param_rok-produkcji__int', 'param_moc__int', 'param_skrzynia-biegów__cat', 'feature_kamera-cofania__cat', 'param_marka-pojazdu__cat', 'param_pojemność-skokowa__int', 'feature_bluetooth__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_klimatyzacja-manualna__cat', 'param_kod-silnika__cat', 'feature_gniazdo-aux__cat', 'feature_klimatyzacja-automatyczna__cat', 'feature_radio-fabryczne__cat', 'feature_czujniki-parkowania-tylne__cat', 'feature_poduszki-boczne-tylne__cat', 'feature_odtwarzacz-dvd__cat']

In [31]:
xgb_params = {
    'max_depth': 5,
    'n_esimators': 50,
    'learning_rate': 0.1,
    'seed': 0,
    'objective': 'reg:squarederror',
    'nthread': -1
}

run_model(xgb.XGBRegressor(**xgb_params), feats) #(-8221.313056568333, 49.7301118662453)

(-9281.896007734778, 95.28842284805155)

In [0]:
feats = ['param_moc__int', 
         'param_rok-produkcji__int',
         'param_pojemność-skokowa__int',
         'param_przebieg__int',
         'param_napęd__cat', 
         'param_faktura-vat__cat',
         'param_stan__cat', 
         'feature_kamera-cofania__cat', 
         'param_skrzynia-biegów__cat', 
         'param_marka-pojazdu__cat', 
         'feature_bluetooth__cat', 
         'feature_łopatki-zmiany-biegów__cat', 
         'feature_światła-led__cat', 
         'feature_klimatyzacja-manualna__cat', 
         'param_kod-silnika__cat', 
         'feature_gniazdo-aux__cat', 
         'feature_klimatyzacja-automatyczna__cat', 
         'feature_radio-fabryczne__cat', 
         'feature_czujniki-parkowania-tylne__cat', 
         'feature_poduszki-boczne-tylne__cat', 
         'feature_odtwarzacz-dvd__cat']

In [0]:
feats_big = ['param_moc__int', 
            'param_rok-produkcji__int',
            'param_pojemność-skokowa__int',
            'param_przebieg__int', 
            'created_at__cat',
            'seller_address__cat',
            'seller_name__cat',
            'seller_type__cat',
            'feature_czujniki-parkowania-przednie__cat',
            'feature_poduszka-powietrzna-chroniąca-kolana__cat',
            'feature_kurtyny-powietrzne__cat',
            'feature_klimatyzacja-dwustrefowa__cat',
            'feature_światła-led__cat',
            'feature_czujnik-zmierzchu__cat',
            'feature_elektrycznie-ustawiane-lusterka__cat',
            'feature_asr-(kontrola-trakcji)__cat',
            'feature_poduszka-powietrzna-kierowcy__cat',
            'feature_cd__cat',
            'feature_elektryczne-szyby-przednie__cat',
            'feature_poduszka-powietrzna-pasażera__cat',
            'feature_system-start-stop__cat',
            'feature_światła-do-jazdy-dziennej__cat',
            'feature_komputer-pokładowy__cat',
            'feature_elektryczne-szyby-tylne__cat',
            'feature_klimatyzacja-manualna__cat',
            'feature_tapicerka-welurowa__cat',
            'feature_czujnik-deszczu__cat',
            'feature_światła-przeciwmgielne__cat',
            'feature_ogrzewanie-postojowe__cat',
            'feature_radio-niefabryczne__cat',
            'feature_regulowane-zawieszenie__cat',
            'feature_ogranicznik-prędkości__cat',
            'feature_zmieniarka-cd__cat',
            'feature_szyberdach__cat',
            'feature_isofix__cat',
            'feature_asystent-pasa-ruchu__cat',
            'feature_alufelgi__cat',
            'feature_bluetooth__cat',
            'feature_nawigacja-gps__cat',
            'feature_asystent-parkowania__cat',
            'feature_wspomaganie-kierownicy__cat',
            'feature_podgrzewana-przednia-szyba__cat',
            'feature_przyciemniane-szyby__cat',
            'feature_elektrycznie-ustawiane-fotele__cat',
            'feature_klimatyzacja-czterostrefowa__cat',
            'feature_tuner-tv__cat',
            'feature_poduszki-boczne-przednie__cat',
            'feature_tempomat-aktywny__cat',
            'feature_klimatyzacja-automatyczna__cat',
            'feature_poduszki-boczne-tylne__cat',
            'feature_radio-fabryczne__cat',
            'feature_odtwarzacz-dvd__cat',
            'feature_czujniki-parkowania-tylne__cat',
            'feature_łopatki-zmiany-biegów__cat',
            'feature_centralny-zamek__cat',
            'feature_dach-panoramiczny__cat',
            'feature_kamera-cofania__cat',
            'feature_immobilizer__cat',
            'feature_czujnik-martwego-pola__cat',
            'feature_gniazdo-sd__cat',
            'feature_podgrzewane-lusterka-boczne__cat',
            'feature_podgrzewane-przednie-siedzenia__cat',
            'feature_hak__cat',
            'feature_abs__cat',
            'feature_wielofunkcyjna-kierownica__cat',
            'feature_światła-xenonowe__cat',
            'feature_gniazdo-usb__cat',
            'feature_tapicerka-skórzana__cat',
            'feature_gniazdo-aux__cat',
            'feature_esp-(stabilizacja-toru-jazdy)__cat',
            'feature_alarm__cat',
            'feature_podgrzewane-tylne-siedzenia__cat',
            'feature_mp3__cat',
            'feature_tempomat__cat',
            'feature_hud-(wyświetlacz-przezierny)__cat',
            'feature_elektrochromatyczne-lusterko-wsteczne__cat',
            'feature_relingi-dachowe__cat',
            'feature_elektrochromatyczne-lusterka-boczne__cat',
            'param_liczba-miejsc__cat',
            'param_uszkodzony__cat',
            'param_marka-pojazdu__cat',
            'param_model__cat',
            'param_engine-code__cat',
            'param_liczba-drzwi__cat',
            'param_first-registration__cat',
            'param_country-of-origin__cat',
            'param_pierwsza-rejestracja__cat',
            'param_gearbox__cat',
            'param_faktura-vat__cat',
            'param_vin__cat',
            'param_perłowy__cat',
            'param_homologacja-ciężarowa__cat',
            'param_service-record__cat',
            'param_metallic__cat',
            'param_leasing-concession__cat',
            'param_color__cat',
            'param_financing-option__cat',
            'param_original-owner__cat',
            'param_vat-marża__cat',
            'param_kategoria__cat',
            'param_co2-emissions__cat',
            'param_leasing__cat',
            'param_mileage__cat',
            'param_zarejestrowany-jako-zabytek__cat',
            'param_napęd__cat',
            'param_wartość-wykupu__cat',
            'param_typ__cat',
            'param_metalik__cat',
            'param_pierwszy-właściciel__cat',
            'param_skrzynia-biegów__cat',
            'param_vat-discount__cat',
            'param_no-accident__cat',
            'param_liczba-pozostałych-rat__cat',
            'param_particle-filter__cat',
            'param_zarejestrowany-w-polsce__cat',
            'param_oferta-od__cat',
            'param_make__cat',
            'param_kraj-pochodzenia__cat',
            'param_engine-power__cat',
            'param_year__cat',
            'param_serwisowany-w-aso__cat',
            'param_rodzaj-paliwa__cat',
            'param_możliwość-finansowania__cat',
            'param_registered-in-poland__cat',
            'param_transmission__cat',
            'param_miesięczna-rata__cat',
            'param_kod-silnika__cat',
            'param_version__cat',
            'param_opłata-początkowa__cat',
            'param_model-pojazdu__cat',
            'param_kierownica-po-prawej-(anglik)__cat',
            'param_fuel-type__cat',
            'param_vat-free__cat',
            'param_acrylic__cat',
            'param_damaged__cat',
            'param_door-count__cat',
            'param_tuning__cat',
            'param_filtr-cząstek-stałych__cat',
            'param_nr-of-seats__cat',
            'param_kolor__cat',
            'param_engine-capacity__cat',
            'param_pearl__cat',
            'param_stan__cat',
            'param_wersja__cat',
            'param_emisja-co2__cat',
            'param_body-type__cat',
            'param_matowy__cat',
            'param_bezwypadkowy__cat',
            'param_akryl-(niemetalizowany)__cat',
            'param_monthly-payment-value__cat']

Hyperopt

In [38]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats_big)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}


#space
xgb_reg_params = {
      'learning_rate':    hp.choice('learning_rate', np.arange(0.01, 0.35, 0.01)),
      'max_depth':        hp.choice('max_depth',     np.arange(8, 16, 1, dtype=int)),
      'subsample':        hp.quniform('subsample', 0.5, 1, 0.01),
      'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.01),
      'objective':        'reg:squarederror',
      'n_estimators':     100,
      'seed': 0,
      #'nthread': 2,
}

#run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)
best

Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.18000000000000002, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.77}
Training with params: 
{'colsample_bytree': 0.6, 'learning_rate': 0.15000000000000002, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.64}
Training with params: 
{'colsample_bytree': 0.52, 'learning_rate': 0.13, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.77}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.33, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training with params: 
{'colsample_bytree': 0.85, 'learning_rate': 0.28, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.93}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.15000000000000002, 'max_depth': 13

{'colsample_bytree': 0.67,
 'learning_rate': 14,
 'max_depth': 3,
 'subsample': 1.0}